In [9]:
%%time
import os
import json
import datetime
from sys import getsizeof
import numpy as np
from scipy import signal
from IPython.display import Audio

import cudf
import cuspatial
import cupy as cp
import cusignal

print(cuspatial.point_in_polygon_bitmap)
print(cudf.__version__)

<function point_in_polygon_bitmap at 0x7f8a41f3d378>
0.13.0
CPU times: user 349 µs, sys: 31 µs, total: 380 µs
Wall time: 326 µs


In [2]:
def gpu_gen_signal(fs, freq, N):
    T = 1/fs
    sig = 0
    x = cp.linspace(0.0, N*(1.0/fs), N)
    for f in freq:
        sig += cp.cos(f*2*cp.pi*x)
    return sig

In [3]:
# Storing num carriers for deep learning prediction -- We're even HURTING ourself here with benchmarks!
def gpu_gen_ensemble(fs, N, num_sig):
    sig_ensemble = cp.zeros((int(num_sig), int(N)))
    num_carriers = cp.zeros(int(num_sig))
    for i in range(int(num_sig)):
        # random number of carriers in random locations for each signal
        num_carrier = int(cp.random.randint(1,5))
        freq = 1e6 * cp.random.randint(1, 10, num_carrier)
        sig_ensemble[i,:] = gpu_gen_signal(fs, freq, N)
        num_carriers[i] = num_carrier
    return sig_ensemble, num_carriers

In [4]:
# 10MHz
fs = 10e6

# Overwrite
num_sig = 2000
N = 2**15

# Change sample rate so N=2^16
up = 2
down = 2

In [5]:
%%time
[gpu_ensemble, num_carriers] = gpu_gen_ensemble(fs, N, num_sig)

CPU times: user 5 s, sys: 6.5 ms, total: 5.01 s
Wall time: 5.01 s


In [6]:
%%time
resample_gpu_ensemble = cusignal.resample_poly(gpu_ensemble, up, down, axis=1, window='flattop')

CPU times: user 2.98 ms, sys: 71 µs, total: 3.05 ms
Wall time: 2.56 ms


In [7]:
resample_gpu_ensemble[2][5]

array(1.99999999)